In [2]:
import pandas as pd
from glob import glob
import os
import ast # for converting string to literal representation
import re
import datetime
#from tqdm import tqdm
import sqlite3

In [3]:
def reform_posts(indf):
    df = indf.copy()
    remove_cols = ['Unnamed: 0', 'available', 'factcheck',
        'image', 'image_id', 'image_ids', 'image_lowquality',
        'images', 'images_description', 'images_lowquality',
        'images_lowquality_description', 'is_live',
        'reactions', 'reactors',
        'sharers', 'shares', 'video', 'video_duration_seconds', 'video_height',
        'video_id', 'video_ids', 'video_quality', 'video_size_MB',
        'video_thumbnail', 'video_watches', 'video_width', 'videos',
        'w3_fb_url', 'was_live', 'with']
    for col in remove_cols:
        if col in df.columns:
            df = df.drop(columns=col)
    df["comments_full"] = df["comments_full"].fillna("[]")
    df["comments_full"] = df["comments_full"].apply(reform_comment)
    # remove duplicates based on post_id--keep row with highest comment count
    df.sort_values(["comments"], inplace=True, ascending=False)
    df = df.groupby(["post_id"]).aggregate('first').reset_index()
    return df

In [4]:
def reform_comment(text):
    date_re = re.compile("datetime.datetime(\(.*?\))")
    dates = re.findall("datetime.datetime\(.*?\)", text)
    for date in dates:
        time_obj = eval(date)
        formatted = "'{}-{}-{} {}:{}:{}'".format(time_obj.year,time_obj.month,time_obj.day,time_obj.hour,time_obj.minute,time_obj.second)
        text = text.replace(date,formatted)
    text = text.replace("None","[]")
    return ast.literal_eval(text)

In [5]:
def get_comments_replies(df):
    commentsdf = pd.DataFrame()
    repliesdf = pd.DataFrame()
    for idx, row in df.iterrows():
        for comment in row["comments_full"]:
            if "replies" in comment:
                for reply in comment["replies"]:
                    reply["parent_comment_id"] = comment["comment_id"]
                    repliesdf = repliesdf.append(reply,ignore_index=True)
            comment.pop("replies",None)
            comment["parent_post_id"] = row["post_id"]
            commentsdf = commentsdf.append(comment, ignore_index = True)
    # trim comment/reply duplicates
    commentsdf = commentsdf.groupby(["comment_id"]).aggregate('first').reset_index()
    repliesdf = repliesdf.groupby(["comment_id"]).aggregate('first').reset_index()
    return commentsdf, repliesdf

In [6]:
datadir = "C:\\Users\\keatu\\Regis_archive\\practicum_data\\raw\\covid19-survivors-story-after-effects"
csv_list = glob(os.path.join(datadir,"*.csv"))

In [8]:
all_posts = []
all_comments = []
all_replies = []
for i, csv in enumerate(csv_list):
    print("{} of {}: {}".format(i+1, len(csv_list),csv))
    fbdf = pd.read_csv(csv)
    fbdf = reform_posts(fbdf)
    commentsdf, repliesdf = get_comments_replies(fbdf)
    fbdf.drop(columns = ["comments_full"])
    all_posts.append(fbdf)
    all_comments.append(commentsdf)
    all_replies.append(repliesdf)
posts = pd.concat(all_posts, sort=False)
comments = pd.concat(all_comments, sort=False)
replies = pd.concat(all_replies, sort=False)

1 of 7: C:\Users\keatu\Regis_archive\practicum_data\raw\covid19-survivors-story-after-effects\fb_scraper_2022-2-11_22-9-48.csv
2 of 7: C:\Users\keatu\Regis_archive\practicum_data\raw\covid19-survivors-story-after-effects\fb_scraper_2022-2-11_23-10-1.csv
3 of 7: C:\Users\keatu\Regis_archive\practicum_data\raw\covid19-survivors-story-after-effects\fb_scraper_2022-2-12_12-37-10.csv
4 of 7: C:\Users\keatu\Regis_archive\practicum_data\raw\covid19-survivors-story-after-effects\fb_scraper_2022-2-12_13-38-7.csv
5 of 7: C:\Users\keatu\Regis_archive\practicum_data\raw\covid19-survivors-story-after-effects\fb_scraper_2022-2-12_14-43-25.csv
6 of 7: C:\Users\keatu\Regis_archive\practicum_data\raw\covid19-survivors-story-after-effects\fb_scraper_2022-2-12_8-36-35.csv
7 of 7: C:\Users\keatu\Regis_archive\practicum_data\raw\covid19-survivors-story-after-effects\fb_scraper_2022-2-12_9-36-47.csv


In [9]:
print(len(posts),len(comments), len(replies))

955 5465 4776


In [33]:
outcon = sqlite3.connect(r"C:\Users\keatu\Regis_archive\practicum_data\Facebook.db")
posts.astype(str).to_sql("posts",con=outcon)
comments.astype(str).to_sql("comments",con=outcon)
replies.astype(str).to_sql("replies",con=outcon)
outcon.close()

In [34]:
print(len(posts),len(comments),len(replies))

3815 125153 239957
